In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [2]:
pos_df = pd.read_csv('WRKY_info_20190507/WRKY_info_table_positive.csv', sep='\t')
neg1_df = pd.read_csv('WRKY_info_20190507/WRKY_info_table_negative_one.csv', sep='\t')
neg2_df = pd.read_csv('WRKY_info_20190507/WRKY_info_table_negative_two.csv', sep='\t')
neg3_df = pd.read_csv('WRKY_info_20190507/WRKY_info_table_negative_three.csv', sep='\t')

In [3]:
pos_df['is_combined'] = np.ones(len(pos_df)).astype(int)
neg1_df['is_combined'] = np.zeros(len(pos_df)).astype(int)
neg2_df['is_combined'] = np.zeros(len(pos_df)).astype(int)
neg3_df['is_combined'] = np.zeros(len(pos_df)).astype(int)

In [4]:
print(pos_df.info())
pos_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26480 entries, 0 to 26479
Data columns (total 6 columns):
TF_ID          26480 non-null object
Pseq_ID        26480 non-null object
Pseq           26480 non-null object
DBD_seq        26480 non-null object
matrix_ID      26480 non-null object
is_combined    26480 non-null int64
dtypes: int64(1), object(5)
memory usage: 1.2+ MB
None


,TF_ID,Pseq_ID,Pseq,DBD_seq,matrix_ID,is_combined
0,AT1G13960,TFprotseq_12499,MSEKEEAPSTSKSTGAPSRPTLSLPPRPFSEMFFNGGVGFSPGPMT...,ADDGYNWRKYGQKQVKGSEFPRSYYKCTNPGCPVKKKVERSLDGQV...,TF_motif_seq_0270,1
1,AT1G13960,TFprotseq_12499,MSEKEEAPSTSKSTGAPSRPTLSLPPRPFSEMFFNGGVGFSPGPMT...,ADDGYNWRKYGQKQVKGSEFPRSYYKCTNPGCPVKKKVERSLDGQV...,TF_motif_seq_0339,1
2,AT1G13960,TFprotseq_12499,MSEKEEAPSTSKSTGAPSRPTLSLPPRPFSEMFFNGGVGFSPGPMT...,ADDGYNWRKYGQKQVKGSEFPRSYYKCTNPGCPVKKKVERSLDGQV...,TFmatrixID_0449,1
3,AT1G13960,TFprotseq_12499,MSEKEEAPSTSKSTGAPSRPTLSLPPRPFSEMFFNGGVGFSPGPMT...,ADDGYNWRKYGQKQVKGSEFPRSYYKCTNPGCPVKKKVERSLDGQV...,TFmatrixID_0451,1
4,AT1G13960,TFprotseq_12499,MSEKEEAPSTSKSTGAPSRPTLSLPPRPFSEMFFNGGVGFSPGPMT...,ADDGYNWRKYGQKQVKGSEFPRSYYKCTNPGCPVKKKVERSLDGQV...,TFmatrixID_0465,1


In [5]:
ATCG_1D_list = ['A', 'C', 'G', 'T']
ATCG_2D_list = []
ATCG_3D_list = []

for i in range(4):
    for j in range(4):
        ATCG_2D_list.append(ATCG_1D_list[i] + ATCG_1D_list[j])
        for k in range(4):
            ATCG_3D_list.append(ATCG_1D_list[i] + ATCG_1D_list[j] + ATCG_1D_list[k])

## Read matrices

In [6]:
col_names = ['matrix_ID', 'alength', 'width', 'nsites', 'E', 'ATCG_prob_list', 'DNA_seq'] + ATCG_2D_list + ATCG_3D_list
matrices_df = pd.DataFrame(columns=col_names)

In [7]:
matricesFile = open('WRKY_info_20190507/All_matrices.txt', 'r')
for i in range(8):
    matricesFile.readline()

In [8]:
# decode index to ATCG character
def getATCGchar(index):
    if (index == -1):
        return '_'
    elif (index == 0):
        return 'A'
    elif (index == 1):
        return 'C'
    elif (index == 2):
        return 'G'
    elif (index == 3):
        return 'T'

In [9]:
while True:
    # eat empty line
    matricesFile.readline()
    
    # Read matrix header
    header = matricesFile.readline().rstrip('\n')
    
    # If the header have nothing, it means we are at the EOF, then break and stop reading it
    if len(header) == 0:
        break
    header = header.split()[1]
    
    # eat empty line
    matricesFile.readline()
    
    # read matrix basic info(alength, width, nsites, E)
    matrix_info = matricesFile.readline()
    matrix_info = matrix_info.split()
    
    alength = int(matrix_info[3])
    width = int(matrix_info[5])
    nsites = int(matrix_info[7])
    E = int(matrix_info[9])
    
    # read DNA sequence(ATCG probability)
    ATCG_prob_list = np.zeros((width, 4))
    DNA_seq = ''
    for j in range(width):
        nucleotide = matricesFile.readline()
        nucleotide = nucleotide.split()
        
        # store the posibility of ATCG in every position of DNA sequence into ATCG_prob_list
        ATCG_prob_list[j][0] = float(nucleotide[0])
        ATCG_prob_list[j][1] = float(nucleotide[1])
        ATCG_prob_list[j][2] = float(nucleotide[2])
        ATCG_prob_list[j][3] = float(nucleotide[3])
        
        # Find the max posibility in every position, if prob > 0.5 then choose that one. 
        # if no prob is > 0.5, then drop that position and store it as '_', which represents empty
        max_ATCG = -1
        for k in range(4):
            if (ATCG_prob_list[j][k] > 0.5):
                max_ATCG = k
                break
        DNA_seq += getATCGchar(max_ATCG)
    
    # Append each matrix to dataframe
    matrices_df.loc[len(matrices_df)] = [header, alength, width, nsites, E, ATCG_prob_list, DNA_seq] + [0] * (4*4 + 4*4*4)

In [30]:
matricesFile.close()

In [32]:
for i in range(len(matrices_df)):
    for nucleo in ATCG_2D_list + ATCG_3D_list:
        if (nucleo in matrices_df.iloc[i]['DNA_seq']):
            matrices_df.loc[i][nucleo] = 1

In [82]:
# print(matrices_df.info())
matrices_df.head()

,matrix_ID,alength,width,nsites,E,ATCG_prob_list,DNA_seq,AA,AC,AG,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,TF_motif_seq_0001,4,10,1,0,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",AACCTAACCT,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,TF_motif_seq_0002,4,10,1,0,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",AACGCGTGTC,1,1,0,...,0,0,0,0,0,1,0,0,0,0
2,TF_motif_seq_0003,4,10,1,0,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",AAGCGTAAGT,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,TF_motif_seq_0004,4,10,1,0,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",AATAAA_AAA,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TF_motif_seq_0005,4,10,1,0,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",AATGGAAATG,1,0,0,...,0,0,0,0,1,0,0,0,0,0


### Drop useless columns

In [83]:
matrices_df = matrices_df.drop(columns=['nsites', 'E'])

## join pos dataset with matrices_df by matrix_ID

In [84]:
merged_pos_df = pd.merge(pos_df, matrices_df)
merged_pos_df.head(10)

,TF_ID,Pseq_ID,Pseq,DBD_seq,matrix_ID,is_combined,alength,width,ATCG_prob_list,DNA_seq,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,AT1G13960,TFprotseq_12499,MSEKEEAPSTSKSTGAPSRPTLSLPPRPFSEMFFNGGVGFSPGPMT...,ADDGYNWRKYGQKQVKGSEFPRSYYKCTNPGCPVKKKVERSLDGQV...,TF_motif_seq_0270,1,4,5,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",TGAC_,...,0,0,1,0,0,0,0,0,0,0
1,AT1G13960,TFprotseq_12499,MSEKEEAPSTSKSTGAPSRPTLSLPPRPFSEMFFNGGVGFSPGPMT...,LDDGYRWRKYGQKVVKGNPYPRSYYKCTTPGCGVRKHVERAATDPK...,TF_motif_seq_0270,1,4,5,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",TGAC_,...,0,0,1,0,0,0,0,0,0,0
2,AT1G13960,TFprotseq_12500,MSEKEEAPSTSKSTGAPSRPTLSLPPRPFSEMFFNGGVGFSPGPMT...,ADDGYNWRKYGQKQVKGSEFPRSYYKCTNPGCPVKKKVERSLDGQV...,TF_motif_seq_0270,1,4,5,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",TGAC_,...,0,0,1,0,0,0,0,0,0,0
3,AT1G13960,TFprotseq_12500,MSEKEEAPSTSKSTGAPSRPTLSLPPRPFSEMFFNGGVGFSPGPMT...,LDDGYRWRKYGQKVVKGNPYPRSYYKCTTPGCGVRKHVERAATDPK...,TF_motif_seq_0270,1,4,5,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",TGAC_,...,0,0,1,0,0,0,0,0,0,0
4,AT1G18860,TFprotseq_12501,MDEAKEENRRLKSSLSKIKKDFDILQTQYNQLMAKHNEPTKFQSKG...,MNDGCQWRKYGQKIAKGNPCPRAYYRCTIAASCPVRKQVQRCSEDM...,TF_motif_seq_0270,1,4,5,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",TGAC_,...,0,0,1,0,0,0,0,0,0,0
5,AT1G29280,TFprotseq_12502,MKRGLDMARSYNDHESSQETGPESPNSSTFNGMKALISSHSPKRSR...,PSDSWAWRKYGQKPIKGSPYPRGYYRCSSTKGCPARKQVERSRDDP...,TF_motif_seq_0270,1,4,5,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",TGAC_,...,0,0,1,0,0,0,0,0,0,0
6,AT1G29860,TFprotseq_12503,MDDHVEHNYNTSLEEVHFKSLSDCLQSSLVMDYNSLEKVFKFSPYS...,LEDGYRWRKYGQKAVKNSPYPRSYYRCTTQKCNVKKRVERSFQDPS...,TF_motif_seq_0270,1,4,5,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",TGAC_,...,0,0,1,0,0,0,0,0,0,0
7,AT1G30650,TFprotseq_12504,MCSVSELLDMENFQGDLTDVVRGIGGHVLSPETPPSNIWPLPLSHP...,PSDLWAWRKYGQKPIKGSPFPRGYYRCSSSKGCSARKQVERSRTDP...,TF_motif_seq_0270,1,4,5,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",TGAC_,...,0,0,1,0,0,0,0,0,0,0
8,AT1G55600,TFprotseq_12505,MSDFDENFIEMTSYWAPPSSPSPRTILAMLEQTDNGLNPISEIFPQ...,PNDGYRWRKYGQKVVKGNPNPRSYFKCTNIECRVKKHVERGADNIK...,TF_motif_seq_0270,1,4,5,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",TGAC_,...,0,0,1,0,0,0,0,0,0,0
9,AT1G62300,TFprotseq_12506,MDRGWSGLTLDSSSLDLLNPNRISHKNHRRFSNPLAMSRIDEEDDQ...,ISDGCQWRKYGQKMAKGNPCPRAYYRCTMATGCPVRKQVQRCAEDR...,TF_motif_seq_0270,1,4,5,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",TGAC_,...,0,0,1,0,0,0,0,0,0,0


In [85]:
pos_ATCG_sum_dict = merged_pos_df.loc[ : , ATCG_2D_list + ATCG_3D_list].sum()/len(pos_df)

In [86]:
sorted(pos_ATCG_sum_dict.items(), key=lambda d: d[1], reverse=True)

[('GT', 0.992749244712991),
 ('CA', 0.9904456193353475),
 ('AA', 0.9902567975830816),
 ('TC', 0.9902567975830816),
 ('CAA', 0.9902567975830816),
 ('GTC', 0.9902567975830816),
 ('TCA', 0.9902567975830816),
 ('AC', 0.5131797583081571),
 ('AAC', 0.5034365558912387),
 ('GG', 0.19282477341389728),
 ('GGT', 0.19282477341389728),
 ('CG', 0.10898791540785499),
 ('AG', 0.08617824773413897),
 ('AGT', 0.08595166163141994),
 ('ACG', 0.08557401812688822),
 ('TG', 0.03459214501510574),
 ('TGG', 0.02484894259818731),
 ('CT', 0.022734138972809667),
 ('CGG', 0.021790030211480363),
 ('GC', 0.021110271903323263),
 ('CTC', 0.021110271903323263),
 ('GCT', 0.021110271903323263),
 ('TCG', 0.021110271903323263),
 ('GA', 0.00974320241691843),
 ('GAC', 0.00974320241691843),
 ('TGA', 0.00974320241691843),
 ('TT', 0.00615558912386707),
 ('TTG', 0.00615558912386707),
 ('GTT', 0.0024924471299093654),
 ('CGT', 0.0020770392749244714),
 ('AAG', 0.0016238670694864049),
 ('ACT', 0.0016238670694864049),
 ('AAA', 0.001359

In [87]:
plt.figure(figsize=(20,10))
plt.bar(range(len(pos_ATCG_sum_dict)), list(pos_ATCG_sum_dict.values()), align='center')
plt.xticks(range(len(pos_ATCG_sum_dict)), list(pos_ATCG_sum_dict.keys()))
plt.show()

TypeError: 'numpy.ndarray' object is not callable

<Figure size 1440x720 with 0 Axes>

In [88]:
merged_neg1_df = pd.merge(neg1_df, matrices_df)
merged_neg2_df = pd.merge(neg2_df, matrices_df)
merged_neg3_df = pd.merge(neg3_df, matrices_df)

In [89]:
neg1_ATCG_sum_dict = merged_neg1_df.loc[ : , ATCG_2D_list + ATCG_3D_list].sum()/len(neg1_df)
neg2_ATCG_sum_dict = merged_neg2_df.loc[ : , ATCG_2D_list + ATCG_3D_list].sum()/len(neg2_df)
neg3_ATCG_sum_dict = merged_neg3_df.loc[ : , ATCG_2D_list + ATCG_3D_list].sum()/len(neg3_df)

In [90]:
print(sorted(neg1_ATCG_sum_dict.items(), key=lambda d: d[1], reverse=True), end='\n\n')
print(sorted(neg2_ATCG_sum_dict.items(), key=lambda d: d[1], reverse=True), end='\n\n')
print(sorted(neg3_ATCG_sum_dict.items(), key=lambda d: d[1], reverse=True), end='\n\n')

[('CG', 0.4500377643504532), ('AC', 0.4490181268882175), ('AA', 0.42235649546827797), ('CA', 0.4029078549848943), ('GT', 0.3937688821752266), ('TA', 0.3675226586102719), ('TG', 0.3530966767371601), ('AT', 0.3341012084592145), ('TT', 0.3169561933534743), ('CC', 0.2933157099697885), ('GA', 0.2792296072507553), ('GC', 0.2702416918429003), ('TC', 0.25929003021148034), ('GG', 0.2560045317220544), ('AG', 0.24244712990936557), ('ACG', 0.2232250755287009), ('CT', 0.22277190332326283), ('CGT', 0.19830060422960724), ('CAC', 0.18916163141993958), ('TAA', 0.14882930513595166), ('GTG', 0.14773413897280968), ('AAA', 0.138595166163142), ('AAT', 0.13444108761329304), ('CCA', 0.12012839879154079), ('CAA', 0.11944864048338369), ('ACA', 0.11514350453172205), ('AAC', 0.1149546827794562), ('CCG', 0.11461480362537764), ('TTA', 0.11363293051359516), ('ATT', 0.11295317220543806), ('AAG', 0.11068731117824773), ('ACC', 0.11057401812688822), ('ATA', 0.10268126888217523), ('CGG', 0.10158610271903323), ('TGT', 0.1

# K means

In [91]:
#接下來匯入KMeans函式庫
from sklearn.cluster import KMeans

#請KMeans分成三類
clf = KMeans(n_clusters=15)

#開始訓練！
clf.fit(merged_pos_df.loc[ : , ATCG_2D_list + ATCG_3D_list])

#這樣就可以取得預測結果了！
clf.labels_

array([11, 11, 11, ...,  3,  3,  3], dtype=int32)

## mix pos data & neg data together

In [92]:
allData_df = merged_pos_df

In [93]:
allData_df = allData_df.append([merged_neg1_df, merged_neg3_df], ignore_index=True)

# Logistic

In [94]:
from sklearn.model_selection import train_test_split

y = allData_df['is_combined']
X_train, X_test, y_train, y_test = train_test_split(allData_df.drop(columns=['TF_ID', 'Pseq_ID', 'Pseq', 'DBD_seq', 'matrix_ID', 'is_combined', 'ATCG_prob_list', 'DNA_seq']), y, test_size=0.1)

In [95]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
_ = lr_model.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [96]:
lr_predict = lr_model.predict(X_test)
print(accuracy_score(y_test, lr_predict))

0.982754279959718


## drop 16

In [97]:
y = allData_df['is_combined']
X_train, X_test, y_train, y_test = train_test_split(allData_df.drop(columns=['TF_ID', 'Pseq_ID', 'Pseq', 'DBD_seq', 'matrix_ID', 'is_combined', 'ATCG_prob_list', 'DNA_seq'] + ATCG_2D_list), y, test_size=0.1)

In [98]:
lr_model = LogisticRegression()
_ = lr_model.fit(X_train, y_train)
lr_predict = lr_model.predict(X_test)
print(accuracy_score(y_test, lr_predict))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9852719033232629
